In [ ]:
%%capture
import sys

# Añade el directorio principal al path de búsqueda para importar módulos desde esa ubicación
sys.path.insert(0, "..")

import numpy as np
import pygame
import torch
import torch.optim as optim
from IPython.display import clear_output
from PIL import Image
from pygame.locals import *

from likelihood.models.deep import MultiBanditNet
from likelihood.tools import train_model_with_episodes

In [ ]:
def render_with_pygame(env):
    # Inicializar Pygame
    pygame.init()

    DISPLAYSURF = pygame.display.set_mode((625, 400), 0, 32)
    clock = pygame.time.Clock()
    pygame.display.flip()

    def print_summary(text, cood, size):
        font = pygame.font.Font(pygame.font.get_default_font(), size)
        text_surface = font.render(text, True, (125, 125, 125))
        DISPLAYSURF.blit(text_surface, cood)

    done = False
    count = 0
    steps = 0
    done = False
    state, _ = env.reset()
    steps += 1

    while count < 10_000:
        pygame.event.get()
        steps += 1

        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                raise Exception("training ended")
        state_tensor = torch.tensor(
            state[0] if type(state) == tuple else state, dtype=torch.float32
        ).unsqueeze(0)
        option_probs, action_probs, termination_probs, selected_option, action = model(state_tensor)
        next_state, reward, done, truncated, info = env.step(action.item())
        print("state:", state)
        print("action taked:", action.item())
        clear_output(wait=True)

        image = env.render()

        image = Image.fromarray(image, "RGB")
        mode, size, data = image.mode, image.size, image.tobytes()
        image = pygame.image.fromstring(data, size, mode)

        DISPLAYSURF.blit(image, (0, 0))
        print_summary("Step {}".format(steps), (10, 10), 15)
        pygame.display.update()
        clock.tick(10)
        count += 1
        if done:
            print_summary("Episode ended !".format(steps), (100, 100), 30)
            pygame.quit()

            done = False
        state = next_state

    pygame.quit()

In [ ]:
# Initialize environment, model, and optimizer
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="rgb_array")  # Example environment
state_dim = env.observation_space.shape[0]
num_options = 1  # Example: 2 options
num_actions = env.action_space.n  # Number of actions in the environment

num_episodes = 20

model = MultiBanditNet(state_dim, num_options, num_actions)
# Example state
state = torch.tensor(np.random.randn(state_dim), dtype=torch.float32)

# Forward pass through the model
option_probs, action_probs, termination_prob, selected_option, action = model(state)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
model, final_loss = train_model_with_episodes(model, optimizer, env, num_episodes)

In [ ]:
%%capture
render_with_pygame(env)